In [134]:
import cv2
import numpy as np

In [135]:
input_image_file_name_1 = 'in1.jpg'
input_image_file_name_2 = 'in2.jpg'
output_image_file_name_1 = 'out1.jpg'
output_image_file_name_2 = 'out2.jpg'
image_input_1 = None
image_input_2 = None
image_output_1 = None
image_output_2 = None
function = None
window_name_1 = 'output-1'
window_name_2 = 'output-2'
show_track_bar = False

In [136]:
def load_help():
    return np.full((400,800), 255, dtype=np.uint8)

def get_help_string():
    hstr = 'Help : \n'
    hstr += 'h) Print help.\n'
    hstr += 'i) Load input images for processing.\n'
    hstr += 's) Save processed images.\n'
    hstr += 'c) Detet corners without OpenCV.\n'
    hstr += 'C) Detect corners with OpenCV.\n'
    hstr += 'e or <ESC>) Exit from the application.\n'
    return hstr

def print_help():
    global image_input_1, image_input_2, image_output_1, image_output_2
    image_output_1 = np.copy(image_input_1)
    image_output_2 = None
    help_str = get_help_string()
    help_str = help_str.split('\n')
    line_index = 0
    for hstr in help_str:
        line_index += 1
        image_output_1 = cv2.putText(
            image_output_1,
            hstr, 
            (50,line_index*50), 
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0,0,0),
            1
        )
    return

In [137]:
handler_name_noise_1 = 'noise_1'
handler_name_gaussian_variance_1 = 'gaussian_variance_1'
handler_name_window_size_1 = 'window_size_1'
handler_name_k_1 = 'k_1'
handler_name_threshold_1 = 'threshold_1'

handler_name_noise_2 = 'noise_2'
handler_name_gaussian_variance_2 = 'gaussian_variance_2'
handler_name_window_size_2 = 'window_size_2'
handler_name_k_2 = 'k_2'
handler_name_threshold_2 = 'threshold_2'

noise_1 = 0
gaussian_variance_1 = 0
window_size_1 = 0
k_1 = 0
threshold_1 = 0

noise_2 = 0
gaussian_variance_2 = 0
window_size_2 = 0
k_2 = 0
threshold_2 = 0

max_noise = 100
max_smoothing_amount = 5
max_gaussian_variance = (max_smoothing_amount // 5) * 100
max_window_size = 9
max_k = 50
max_threshold = 1000

square_padding = 5

In [138]:
def handler_noise_1(n):
    global noise_1
    if n >= 0:
        noise_1 = n
    return

def handler_gaussian_variance_1(gv):
    global gaussian_variance_1
    if gv >= 0:
        gaussian_variance_1 = gv
    return

def handler_window_size_1(ws):
    global window_size_1
    if ws >= 0:
        window_size_1 = ws
    return

def handler_k_1(kv):
    global k_1
    if kv >= 0:
        k_1 = kv
    return

def handler_threshold_1(thr):
    global threshold_1
    if thr >= 0:
        threshold_1 = thr
    return

def handler_noise_2(n):
    global noise_2
    if n >= 0:
        noise_2 = n
    return

def handler_gaussian_variance_2(gv):
    global gaussian_variance_2
    if gv >= 0:
        gaussian_variance_2 = gv
    return

def handler_window_size_2(ws):
    global window_size_2
    if ws >= 0:
        window_size_2 = ws
    return

def handler_k_2(kv):
    global k_2
    if kv >= 0:
        k_2 = kv
    return

def handler_threshold_2(thr):
    global threshold_2
    if thr >= 0:
        threshold_2 = thr
    return

In [139]:
def load_image(image_file_name):
    image_load = cv2.imread(image_file_name)
    return image_load

def save_image():
    global image_output_1, image_output_2, output_image_file_name_1, output_image_file_name_2
    cv2.imwrite(output_image_file_name_1, image_output_1)
    cv2.imwrite(output_image_file_name_2, image_output_2)
    return

def add_gaussian_noise(image, mean, variance):
    row, col, channel= image.shape
    var = 0.1
    sigma = variance**0.5
    gauss = np.random.normal(mean, sigma, (row, col, channel))
    gauss = gauss.reshape(row, col, channel)
    noisy = image + gauss
    noisy = noisy.astype(np.uint8)
    return noisy

def grayscale_from_color(color_image):
    grayscale_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
    return grayscale_image

def gaussian_smoothing(image, standard_deviation):
    global max_smoothing_amount
    smooth_image = cv2.GaussianBlur(image,(max_smoothing_amount,max_smoothing_amount),standard_deviation,standard_deviation)
    return smooth_image

def grad_x(image):
    deriv_length = 3
#     gradient_x = cv2.Sobel(image,cv2.CV_8U,1,0,ksize=deriv_length)
    kx, ky = cv2.getDerivKernels(1, 0, deriv_length, True)
    filter_x = kx.T
    gradient_x = cv2.filter2D(image, -1, filter_x)
    gradient_x = np.float64(gradient_x)
    gradient_x -= np.amin(gradient_x)
    gradient_x *= (255.0 / np.amax(gradient_x))
    gradient_x = np.uint8(gradient_x)
    return gradient_x

def grad_y(image):
    deriv_length = 3
#     gradient_y = cv2.Sobel(image,cv2.CV_8U,0,1,ksize=deriv_length)
    kx, ky = cv2.getDerivKernels(0, 1, deriv_length, True)
    filter_y = ky
    gradient_y = cv2.filter2D(image, -1, filter_y)
    gradient_y = np.float64(gradient_y)
    gradient_y -= np.amin(gradient_y)
    gradient_y *= (255.0 / np.amax(gradient_y))
    gradient_y = np.uint8(gradient_y)
    return gradient_y

def magnitude_gradient(grad_x, grad_y):
    mag_grad = np.hypot(grad_x,grad_y)
    mag_grad = np.float64(mag_grad)
    mag_grad -= np.amin(mag_grad)
    mag_grad *= (255.0 / np.amax(mag_grad))
    mag_grad = np.uint8(mag_grad)
    return mag_grad

def angle_gradient(grad_x, grad_y):
    angle_grad = np.arctan2(grad_y, grad_x) * 180 / np.pi
    angle_grad = angle_grad % 360
    return angle_grad

In [140]:
def non_maximum_suppression(corner_points, window_size):
    
    corner_points.sort(key = lambda p : p['cornerness'], reverse=True)
    
    nms_corner_points = []
    
    for cp in corner_points:
        is_present = False
        
        for nms_cp in nms_corner_points:
            if (((cp['r'] - nms_cp['r'])**2) + ((cp['c'] - nms_cp['c'])**2)) <= window_size**2:
                is_present = True
                break
        
        if not is_present:
            nms_corner_points.append(cp)
    
    return nms_corner_points

def corner_detection(image, window_size, k, threshold):
    
    corner_points = []
    
    dx, dy = grad_x(image), grad_y(image)
    
    XX = dx**2
    XY = dx*dy
    YX = dy*dx
    YY = dy**2
    
    index = 0
    for row in range(0, image.shape[0], 1):
        for col in range(0, image.shape[1], 1):
            c_00 = XX[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1]
            c_01 = XY[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1]
            c_10 = YX[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1]
            c_11 = YY[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1]
            
            c_00 = c_00.sum()
            c_01 = c_01.sum()
            c_10 = c_10.sum()
            c_11 = c_11.sum()
            
            c = np.array([[c_00, c_01], [c_10, c_11]])
            
            cornerness = (np.linalg.det(c) - (k * (np.trace(c)**2)))
            
            if cornerness > threshold:
                corner_points.append({'id' : index, 'r' : row, 'c' : col, 'cornerness': cornerness})
                index += 1
    
    nms_corner_points = non_maximum_suppression(corner_points, window_size)
    
    p = np.array([[ [r, c] for c in range(image.shape[1])] for r in range(image.shape[0])])
    
    localized_corner_points = []
    
    for nms_cp in nms_corner_points:
        row = nms_cp['r']
        col = nms_cp['c']
        index = nms_cp['id']
        
        c_00 = XX[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1]
        c_01 = XY[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1]
        c_10 = YX[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1]
        c_11 = YY[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1]
        
        pc_0 = c_00 * p[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1, 0] + c_01 * p[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1, 1]
        pc_1 = c_10 * p[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1, 0] + c_11 * p[row-(window_size//2) : row+(window_size//2)+1, col-(window_size//2) : col+(window_size//2)+1, 1]
        
        c_00 = c_00.sum()
        c_01 = c_01.sum()
        c_10 = c_10.sum()
        c_11 = c_11.sum()
        
        pc_0 = pc_0.sum()
        pc_1 = pc_1.sum()
        
        c = np.array([[c_00, c_01], [c_10, c_11]])
        pc = np.array([[pc_0], [pc_1]])
        
        pc = np.matmul(np.linalg.inv(c), pc)
        pc_r = int(pc[0][0])
        pc_c = int(pc[1][0])
        
        localized_corner_points.append({'id': index, 'r' : pc_r, 'c' : pc_c, 'cornerness': nms_cp['cornerness']})
    
    return localized_corner_points

In [141]:
def get_nearest_bin(angle, angle_unit, bin_size):
    angle_unit = 360 / bin_size
    
    index = int(angle//angle_unit)
    mod = angle % angle_unit
    
    if index == bin_size:
        return index-1, index%bin_size, mod
    else:
        return index, (index+1)%bin_size, mod

def extract_cell_hog_features(d_magnitude, d_angle, bin_size):
    features = [0] * bin_size
    
    for row in range(0, d_magnitude.shape[0], 1):
        for col in range(0, d_angle.shape[1], 1):
            feature_strength = d_magnitude[row][col]
            feature_angle = d_angle[row][col]
            angle_unit = 360 / bin_size
            min_angle, max_angle, mod = get_nearest_bin(feature_angle, angle_unit, bin_size)
            features[min_angle] += (feature_strength * (1 - (mod / angle_unit)))
            features[max_angle] += (feature_strength * (mod / angle_unit))
    
    max_index = np.argmax(features)
    features = np.roll(features, bin_size - max_index)
    return np.array(features)

def extract_hog_features(image, points, patch_size, cell_size, bin_size):
    
    hog_features = []
    
    rows, cols = image.shape[:2]
    
    dx, dy = grad_x(image), grad_y(image)
    d_magnitude = magnitude_gradient(dx, dy)
    d_angle = angle_gradient(dx, dy)
    
    for p in points:
        rp = p['r']
        cp = p['c']
        idp = p['id']
        
        patch_r_start, patch_r_end = rp - (patch_size//2), rp + (patch_size//2)
        patch_c_start, patch_c_end = cp - (patch_size//2), cp + (patch_size//2)
        
        d_mag_patch = d_magnitude[patch_r_start:patch_r_end, patch_c_start:patch_c_end]
        d_angle_patch = d_angle[patch_r_start:patch_r_end, patch_c_start:patch_c_end]
        
        cells_features = np.empty(((patch_r_end-patch_r_start)//cell_size, (patch_c_end-patch_c_start)//cell_size), dtype=object)

        for row_c in range(0, cells_features.shape[0], 1):
            cell_r_start = patch_r_start + (row_c * cell_size)
            cell_r_end = cell_r_start + cell_size
            for col_c in range(0, cells_features.shape[1], 1):
                cell_c_start = patch_c_start + (col_c * cell_size)
                cell_c_end = cell_c_start + cell_size
                
                d_mag_cell = d_magnitude[cell_r_start:cell_r_end, cell_c_start:cell_c_end]
                d_angle_cell = d_angle[cell_r_start:cell_r_end, cell_c_start:cell_c_end]
                
                cells_features[row_c][col_c] = extract_cell_hog_features(d_mag_cell, d_angle_cell, bin_size)
        
        hog_features.append({'id' : idp, 'r' : rp, 'c' : cp, 'ftrs' : cells_features})
    
    return hog_features

In [142]:
def euclidean_distance(vector_1, vector_2):
    return np.linalg.norm(vector_1-vector_2)

def compare_features(features_1, features_2):
    distance = 0
    for r in range(0, features_1.shape[0], 1):
        for c in range(0, features_2.shape[1], 1):
            vector_1 = features_1[r][c]
            vector_2 = features_2[r][c]
            distance += euclidean_distance(vector_1, vector_2)
    return distance

def compare_hog_features(features_1, features_2):
    compared = []
    
    for i2 in range(0, len(features_2), 1):
        ftr2_map = features_2[i2]
        id2 = ftr2_map['id']
        rf2 = ftr2_map['r']
        cf2 = ftr2_map['c']
        ftrs2 = ftr2_map['ftrs']
        
        distances = []
        
        for i1 in range(0, len(features_1), 1):
            ftr1_map = features_1[i1]
            id1 = ftr1_map['id']
            rf1 = ftr1_map['r']
            cf1 = ftr1_map['c']
            ftrs1 = ftr1_map['ftrs']
            
            distances.append(compare_features(ftrs1, ftrs2))
            
        if len(distances) > 0:
            min_index = np.argmin(distances)
            compared.append({'ftrs1': features_1[min_index], 'ftrs2': features_2[i2]})
        
    return compared

In [143]:
def draw_on_image(image_output_1_f, localized_corner_points_1, window_size_1_f, image_output_2_f, localized_corner_points_2, window_size_2_f, square_padding, compared_features):
    numbered_comparison = {}
    
    for cf in compared_features:
        ftr1 = cf['ftrs1']
        ftr2 = cf['ftrs2']
        
        id1 = ftr1['id']
        id2 = ftr2['id']
        
        ftr1_map = numbered_comparison.get(id1, {})
        ftr1_map['ftr1'] = ftr1
        ftr1_map_list = ftr1_map.get('ftr2', [])
        ftr1_map_list.append(ftr2)
        ftr1_map['ftr2'] = ftr1_map_list
        numbered_comparison[id1] = ftr1_map
    
    for lcp in localized_corner_points_1:
        cv2.rectangle(
            image_output_1_f,
            (lcp['c'] - (window_size_1_f//2) - square_padding, lcp['r'] - (window_size_1_f//2) - square_padding),
            (lcp['c'] + (window_size_1_f//2) + square_padding, lcp['r'] + (window_size_1_f//2) + square_padding),
            (0,0,255),
            2
        )
    
    for lcp in localized_corner_points_2:
        cv2.rectangle(
            image_output_2_f,
            (lcp['c'] - (window_size_2_f//2) - square_padding, lcp['r'] - (window_size_2_f//2) - square_padding),
            (lcp['c'] + (window_size_2_f//2) + square_padding, lcp['r'] + (window_size_2_f//2) + square_padding),
            (0,0,255),
            2
        )
    
    index = 1
    for ftr1_key, ftr1_map in numbered_comparison.items():
        cv2.putText(
            image_output_1_f,
            f'{index}', 
            (numbered_comparison[ftr1_key]['ftr1']['c'], numbered_comparison[ftr1_key]['ftr1']['r']), 
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0,255,0),
            2
        )
        for ftr2 in ftr1_map['ftr2']:
            cv2.putText(
                image_output_2_f,
                f'{index}', 
                (ftr2['c'], ftr2['r']), 
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0,255,0),
                2
            )
        index += 1
        
    return image_output_1_f, image_output_2_f

In [144]:
def harris_corner_detection_impl():
    global image_input_1, image_input_2, image_output_1, image_output_2, noise_1, gaussian_variance_1, window_size_1, k_1, threshold_1, noise_2, gaussian_variance_2, window_size_2, k_2, threshold_2
    
    noise_1_f = noise_1 / 100
    gaussian_variance_1_f = gaussian_variance_1 / 100
    window_size_1_f = (2 * window_size_1) + 1
    k_1_f = k_1 / 100
    threshold_1_f = threshold_1 * 10000
    
    noise_2_f = noise_2 / 100
    gaussian_variance_2_f = gaussian_variance_2 / 100
    window_size_2_f = (2 * window_size_2) + 1
    k_2_f = k_2 / 100
    threshold_2_f = threshold_2 * 10000
    
    bin_size_f = 8
    cell_size_f = 6
    patch_size_f = 3 * cell_size_f
     
    image_input_1_f = add_gaussian_noise(image_input_1, 0, noise_1_f)
    image_input_2_f = add_gaussian_noise(image_input_2, 0, noise_2_f)
    image_output_1 = np.copy(image_input_1_f)
    image_output_2 = np.copy(image_input_2_f)
    garyscale_image_input_1 = grayscale_from_color(image_input_1_f)
    garyscale_image_input_2 = grayscale_from_color(image_input_2_f)
    smooth_image_1 = gaussian_smoothing(garyscale_image_input_1, gaussian_variance_1_f)
    smooth_image_2 = gaussian_smoothing(garyscale_image_input_2, gaussian_variance_2_f)
    image_output_1_f = np.copy(smooth_image_1)
    image_output_2_f = np.copy(smooth_image_2)
    
    localized_corner_points_1 = corner_detection(image_output_1_f, window_size_1_f, k_1_f, threshold_1_f)
    localized_corner_points_2 = corner_detection(image_output_2_f, window_size_2_f, k_2_f, threshold_2_f)
    
    features_1 = extract_hog_features(image_output_1_f, localized_corner_points_1, patch_size_f, cell_size_f, bin_size_f)
    features_2 = extract_hog_features(image_output_2_f, localized_corner_points_2, patch_size_f, cell_size_f, bin_size_f)
        
    compared_features = compare_hog_features(features_1, features_2)
    
    image_output_1, image_output_2 = draw_on_image(image_output_1, localized_corner_points_1, window_size_1_f, image_output_2, localized_corner_points_2, window_size_2_f, square_padding, compared_features)

In [145]:
def harris_corner_detection_opencv():
    global image_input_1, image_input_2, image_output_1, image_output_2, noise_1, gaussian_variance_1, window_size_1, k_1, threshold_1, noise_2, gaussian_variance_2, window_size_2, k_2, threshold_2
    
    noise_1_f = noise_1 / 100
    gaussian_variance_1_f = gaussian_variance_1 / 100
    window_size_1_f = (2 * window_size_1) + 1
    k_1_f = k_1 / 100
    threshold_1_f = threshold_1
    
    noise_2_f = noise_2 / 100
    gaussian_variance_2_f = gaussian_variance_2 / 100
    window_size_2_f = (2 * window_size_2) + 1
    k_2_f = k_2 / 100
    threshold_2_f = threshold_2
    
    bin_size_f = 8
    cell_size_f = 6
    patch_size_f = 3 * cell_size_f
     
    image_input_1_f = add_gaussian_noise(image_input_1, 0, noise_1_f)
    image_input_2_f = add_gaussian_noise(image_input_2, 0, noise_2_f)
    image_output_1 = np.copy(image_input_1_f)
    image_output_2 = np.copy(image_input_2_f)
    garyscale_image_input_1 = grayscale_from_color(image_input_1_f)
    garyscale_image_input_2 = grayscale_from_color(image_input_2_f)
    smooth_image_1 = gaussian_smoothing(garyscale_image_input_1, gaussian_variance_1_f)
    smooth_image_2 = gaussian_smoothing(garyscale_image_input_2, gaussian_variance_2_f)
    image_output_1_f = np.copy(smooth_image_1)
    image_output_2_f = np.copy(smooth_image_2)
    
    image_output_1_f = np.float32(image_output_1_f)
    dst_1 = cv2.cornerHarris(image_output_1_f,window_size_1_f,3,k_1_f)
    dst_norm_1 = np.empty(dst_1.shape, dtype=np.float32)
    cv2.normalize(dst_1, dst_norm_1, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    dst_norm_scaled_1 = cv2.convertScaleAbs(dst_norm_1)
    locations_1 = []
    for i in range(dst_norm_1.shape[0]):
        for j in range(dst_norm_1.shape[1]):
            if int(dst_norm_1[i,j]) > threshold_1_f:
                locations_1.append((j, i))
                cv2.rectangle(
                    image_output_1,
                    (j - (window_size_1_f//2) - square_padding, i - (window_size_1_f//2) - square_padding),
                    (j + (window_size_1_f//2) + square_padding, i + (window_size_1_f//2) + square_padding),
                    (0,0,255),
                    2
                )
    
    image_output_2_f = np.float32(image_output_2_f)
    dst_2 = cv2.cornerHarris(image_output_2_f,window_size_2_f,3,k_2_f)
    dst_norm_2 = np.empty(dst_2.shape, dtype=np.float32)
    cv2.normalize(dst_2, dst_norm_2, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    dst_norm_scaled_2 = cv2.convertScaleAbs(dst_norm_2)
    locations_2 = []
    for i in range(dst_norm_2.shape[0]):
        for j in range(dst_norm_2.shape[1]):
            if int(dst_norm_2[i,j]) > threshold_2_f:
                locations_2.append((j, i))
                cv2.rectangle(
                    image_output_2,
                    (j - (window_size_2_f//2) - square_padding, i - (window_size_2_f//2) - square_padding),
                    (j + (window_size_2_f//2) + square_padding, i + (window_size_2_f//2) + square_padding),
                    (0,0,255),
                    2
                )
    
#     hog_1 = cv2.HOGDescriptor((image_output_1.shape[0], image_output_1.shape[1]),(patch_size_f, patch_size_f),(cell_size_f,cell_size_f),(cell_size_f,cell_size_f),bin_size_f,1,-1,0,0.2,True,64)
#     hist_1 = hog_1.compute(image_output_1,(cell_size_f,cell_size_f),(0,0),locations_1)
#     hog_2 = cv2.HOGDescriptor((image_output_2.shape[0], image_output_2.shape[1]),(patch_size_f, patch_size_f),(cell_size_f,cell_size_f),(cell_size_f,cell_size_f),bin_size_f,1,-1,0,0.2,True,64)
#     hist_2 = hog_2.compute(image_output_2,(cell_size_f,cell_size_f),(0,0),locations_2)


In [146]:
def init_app():
    global window_name_1, window_name_2, image_input_1, image_input_2, image_output_1, image_output_2, function, input_image_file_name_1, input_image_file_name_2, output_image_file_name_1, output_image_file_name_2, noise_1, gaussian_variance_1, window_size_1, k_1, threshold_1, noise_2, gaussian_variance_2, window_size_2, k_2, threshold_2
    cv2.destroyAllWindows()
    window_name_1 = 'output-1'
    window_name_2 = 'output-2'
    image_input_1 = None
    image_input_2 = None
    image_output_1 = None
    image_output_2 = None
    function = None
    input_image_file_name_1 = 'in1.jpg'
    input_image_file_name_2 = 'in2.jpg'
    output_image_file_name_1 = 'out1.jpg'
    output_image_file_name_2 = 'out2.jpg'
    noise_1 = 0
    gaussian_variance_1 = 0
    window_size_1 = 0
    k_1 = 0
    threshold_1 = 0
    noise_2 = 0
    gaussian_variance_2 = 0
    window_size_2 = 0
    k_2 = 0
    threshold_2 = 0
    return

def init_help():
    global image_input_1, image_input_2, image_output_1, image_output_2, function, show_track_bar
    cv2.destroyAllWindows()
    image_input_1 = load_help()
    image_input_2 = None
    image_output_1 = np.copy(image_input_1)
    image_output_2 = None
    function = print_help
    show_track_bar = False
    return

def init_load():
    global input_image_file_name_1, input_image_file_name_2, image_input_1, image_input_2, image_output_1, image_output_2, function, show_track_bar
    cv2.destroyAllWindows()
    image_input_1 = load_image(input_image_file_name_1)
    image_input_2 = load_image(input_image_file_name_2)
    image_output_1 = np.copy(image_input_1)
    image_output_2 = np.copy(image_input_2)
    function = None
    show_track_bar = True
    return

def init_cd_c():
    global function, show_track_bar
    cv2.destroyAllWindows()
    function = harris_corner_detection_impl
    show_track_bar = True
    return

def init_cd_C():
    global function, show_track_bar
    cv2.destroyAllWindows()
    function = harris_corner_detection_opencv
    show_track_bar = True
    return

In [152]:
init_app()
init_help()

while(True):
    
    if image_output_1 is not None:
        cv2.namedWindow(window_name_1)
    if image_output_2 is not None:
        cv2.namedWindow(window_name_2)
    
    key = cv2.waitKey(1000) & 0xFF
    
    if key == 27 or key == ord('e'):
        init_app()
        break
    
    if key == ord('h'):
        print(get_help_string())
        init_help()
    
    if key == ord('i'):
        init_load()
    
    if image_input_1 is not None and image_input_2 is not None:
        
        if key == ord('s'):
            save_image()
        
        if key == ord('c'):
            init_cd_c()
        
        if key == ord('C'):
            init_cd_C()
    
    if function is not None:
        function()
        function = None
    
    if show_track_bar:
        cv2.createTrackbar(handler_name_noise_1, window_name_1, noise_1, max_noise, handler_noise_1)
        cv2.createTrackbar(handler_name_gaussian_variance_1, window_name_1, gaussian_variance_1, max_gaussian_variance, handler_gaussian_variance_1)
        cv2.createTrackbar(handler_name_window_size_1, window_name_1, window_size_1, max_window_size, handler_window_size_1)
        cv2.createTrackbar(handler_name_k_1, window_name_1, k_1, max_k, handler_k_1)
        cv2.createTrackbar(handler_name_threshold_1, window_name_1, threshold_1, max_threshold, handler_threshold_1)
        
        cv2.createTrackbar(handler_name_noise_2, window_name_2, noise_2, max_noise, handler_noise_2)
        cv2.createTrackbar(handler_name_gaussian_variance_2, window_name_2, gaussian_variance_2, max_gaussian_variance, handler_gaussian_variance_2)
        cv2.createTrackbar(handler_name_window_size_2, window_name_2, window_size_2, max_window_size, handler_window_size_2)
        cv2.createTrackbar(handler_name_k_2, window_name_2, k_2, max_k, handler_k_2)
        cv2.createTrackbar(handler_name_threshold_2, window_name_2, threshold_2, max_threshold, handler_threshold_2)
    
    if image_output_1 is not None:
        cv2.imshow(window_name_1, image_output_1)
    if image_output_2 is not None:
        cv2.imshow(window_name_2, image_output_2)